# H&M Group

**Разработка рекомендации по продуктам на основе данных предыдущих транзакций, а также метаданных о клиентах и продуктах**

**Цель проекта** — сделать точные предсказания какие товары каждый клиент приобретет в течение 7-дневного периода сразу после окончания тренировочных данных. Клиенты, которые не совершили никаких покупок в течение этого времени, исключаются из оценки.

**Описание данных**

- images/ - папка с изображениями, соответствующими каждому article_id
- article_id имеют соответствующее изображение.
- 
articles.csv - подробная информация о каждом article_id, доступном для покупк
- 
customers.csv - метаданные для каждого customer_id в наборе данн
- х
sample_submission.csv - образец файла для представления результатов в правильном форм
- те
transactions_train.csv - тренировочные данные, состоящие из покупок каждого клиента на каждую дату, а также дополнител информации
- Дублирующиеся строки соответствуют нескольким покупкам одного и того же товара.ода.

## Загрузка данных 

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('C:\\Users\\hp\\Documents\\Учеба IT\\data h and m\\transactions_train.csv')

In [3]:
# Переобразуем столбцы в DataFrame в нужный тип данных
train['customer_id'] = train['customer_id'].str[-16:].apply(lambda x: int(x, 16)).astype('int64')
train['article_id'] = train.article_id.astype('int32')
train.t_dat = pd.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id']]
train.to_parquet('train.pqt',index=False)
print( train.shape )
train.head()

(31788324, 3)


,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,663713001
1,2018-09-20,-6846340800584936,541518023
2,2018-09-20,-8334631767138808638,505221004
3,2018-09-20,-8334631767138808638,685687003
4,2018-09-20,-8334631767138808638,685687004


In [4]:
#Сгруппируем, вычислим разницу в днях между максимальной датой ('max_dat') и датой ('t_dat') 
#для каждой строки в DataFrame 'train' и выведем нужные данные 
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (5181535, 5)


In [5]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
1158548,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
80218,2018-10-04,4485518665254175540,557247001,2018-10-04,0,86
2149868,2020-03-06,-906958334866810496,852521001,2020-03-06,0,81
3423153,2020-07-06,3601599666106972342,685813001,2020-07-06,0,80
871769,2019-05-14,-4601407992705575197,695545001,2019-05-14,0,80


In [6]:
train['article_id'] = ' 0' + train['article_id'].astype('str')
preds = train.groupby('customer_id')['article_id'].sum().reset_index()
preds.columns = ['customer_id', 'prediction']
preds.head()

,customer_id,prediction
0,-9223352921020755230,0673396002 0812167004
1,-9223343869995384291,0908292002 0910601003 0903926002 0865929007
2,-9223321797620987725,0580600006 0610776035
3,-9223319430705797669,0470985003 0562245001 0646592007 0504155001 0...
4,-9223308614576639426,0750423005


In [7]:
#Находим 12 наиболее популярных товаров за последнюю неделю
train = pd.read_parquet('train.pqt')
train['t_dat'] = pd.to_datetime(train['t_dat'])
train = train.loc[train['t_dat'] >= pd.to_datetime('2020-09-16')]
top12 = ' 0'.join(train['article_id'].value_counts().index.astype(str)[:12])
print("Last week's top 12 popular items:")
print(top12)

Last week's top 12 popular items:
924243001 0924243002 0918522001 0923758001 0866731001 0909370001 0751471001 0915529003 0915529005 0448509014 0762846027 0714790020


In [8]:
#Объединим  DataFrame, поменяем на нужный тип данных, заполним пропуски и  выведем нужные данные  
sub = pd.read_csv('C:\\Users\\hp\\Documents\\Учеба IT\\data h and m\\sample_submission.csv')
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].str[-16:].apply(lambda x: int(x, 16)).astype('int64')
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'}, axis=1), on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
sub['prediction'] = sub['prediction'] + top12
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub.to_csv('submission.csv', index=False)
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043924243001 0924243002 0918522001 0923...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002924243001 0924243002 0918522001 0923...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007924243001 0924243002 0918522001 0923...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001924243001 0924243002 0918...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015924...


## Вывод

**Предсказания состоят из комбинации значений из переменной 'top12' и оригинальных предсказаний из DataFrame 'preds'**